In [ ]:
# import model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

# import model from model.h5
from keras.models import load_model, Model
model = load_model('model.h5')

In [2]:
test_df = pd.read_csv("./split-data/test_new.csv")
testgen = ImageDataGenerator(rescale=1./255)
test_generator = testgen.flow_from_dataframe(
    dataframe=test_df,
    directory="./dataset",
    x_col="img_path",
    y_col="category",
    target_size=(256, 256),
    batch_size=128,
    class_mode="categorical",
    workers=4
)

Found 3133 validated image filenames belonging to 46 classes.


In [ ]:
# remove the last layer
model.pop()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy, top_k_categorical_accuracy'])

# save all the features for the test data
features = model.predict_generator(test_generator, workers=4)


In [ ]:
# test using the first element of test_generator
feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)

# Initialize arrays for storing features, inputs, and labels
features = []
inputs = []
labels = []

# Extract features batch by batch
for batch_x, batch_y in test_generator:
    # Stop when we've processed all batches
    if len(features) >= test_generator.samples:
        break
    
    # Extract features for the batch
    batch_features = feature_extractor.predict(batch_x)
    
    # Append features, inputs, and labels
    features.append(batch_features)
    inputs.append(batch_x)
    labels.append(batch_y)

# Convert lists to NumPy arrays
features = np.concatenate(features, axis=0)
inputs = np.concatenate(inputs, axis=0)
labels = np.concatenate(labels, axis=0)

# Save features, inputs, and labels to a file
data_to_save = {
    'features': features,
    'inputs': inputs,
    'labels': labels
}

np.save('clothing_features_from_generator.npy', data_to_save)
